In [2]:
!pip install dash-bootstrap-components


     ------------------------------------ 220.6/220.6 kB 313.1 kB/s eta 0:00:00


In [16]:
import pandas as pd
import dash

import plotly.graph_objects as go
from dash import dcc as dcc
from dash import html as html
from dash.dependencies import Input, Output

import plotly.express as px
from sqlalchemy import create_engine
import dash_bootstrap_components as dbc

# MySQL database settings
db_host = "localhost"
db_user = "root"
db_password = "Shiphrah07"
db_name = "stock"


# Create MySQL engine
db_url = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}"
engine = create_engine(db_url)





# Retrieve data from the database
query = "SELECT * FROM temperature_data"  # Replace with your table name
df = pd.read_sql_query(query, con=engine)

# Get unique thermostat IDs for dropdown options
thermostat_options = [{'label': str(id), 'value': id} for id in df['Thermostat_ID'].unique()]

# Create Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H1("Temperature Dashboard", className="text-center mt-4 mb-4"),
    
    dcc.Dropdown(
        id='chart-type-dropdown',
        options=[
            {'label': 'Line Chart', 'value': 'line'},
            {'label': 'Histogram', 'value': 'histogram'},
            {'label': 'Scatter Plot', 'value': 'scatter'},
            {'label': 'Bar Chart', 'value': 'bar'}
        ],
        value='line'
    ),
    
    dcc.Graph(id="selected-chart"),
    
    dcc.Dropdown(
        id='thermostat-dropdown',
        options=thermostat_options,
        value=df['Thermostat_ID'].unique(),  # Default value (all IDs)
        multi=True
    ),
    
    dcc.Interval(
        id='interval-component',
        interval=60*1000,  # in milliseconds
        n_intervals=0
    )
])

# Callbacks to update the selected chart based on user interaction
@app.callback(
    Output("selected-chart", "figure"),
    [Input("chart-type-dropdown", "value"),
     Input("thermostat-dropdown", "value"),
     Input("interval-component", "n_intervals")]
)
def update_selected_chart(selected_chart, selected_ids, n_intervals):
    filtered_df = df[df['Thermostat_ID'].isin(selected_ids)]
    fig = None
    
    if selected_chart == 'line':
        fig = px.line(filtered_df, x="Timestamp", y="Temperature", color="Thermostat_ID",
                      title="Temperature Readings Over Time")
    elif selected_chart == 'histogram':
        fig = px.histogram(filtered_df, x="Temperature", nbins=20,
                           title="Temperature Distribution",
                           color_discrete_sequence=['#1f77b4'])  # Customize histogram color
    elif selected_chart == 'scatter':
        fig = px.scatter(filtered_df, x="Temperature", y="Thermostat_ID",
                          title="Temperature vs. Thermostat ID",
                          color="Thermostat_ID",
                          color_discrete_sequence=px.colors.qualitative.Plotly)
    elif selected_chart == 'bar':
        fig = px.bar(filtered_df, x="Thermostat_ID", y="Temperature",
                     title="Average Temperature by Thermostat",
                     color="Thermostat_ID",  # Customize bar chart color
                     color_discrete_map={'1': '#1f77b4', '2': '#ff7f0e', '3': '#2ca02c'})  # Customize color mapping
    
    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True, port=4444)
